In [0]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)

In [0]:
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json


#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupiter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
confluentTopicName = "Group3Data"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"


#Kakfa Class Setup.
c = Consumer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,# this will create a new consumer group on each invocation.
    'group.id': str(1),
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': True,
    'error_cb': error_cb,
})

c.subscribe(['Group3Data'])

In [0]:
aString = {}

kafkaList = []

for i in range(1000):
    try:
        msg = c.poll(timeout=15)
        print(msg)
        if msg is None:
            break
        elif msg.error():
            print("Consumer error: {}".format(msg.error()))
            break
        else:
            aString=json.loads('{}'.format(msg.value().decode('utf-8')))
            print('#' + str(i) + ' ' + str(aString))
            kafkaList.append(aString)
            c.commit(asynchronous=False)
    except Exception as e:
        print(e)

<cimpl.Message object at 0x7fd2b8f72f40>
#0 {'DistrictName': 'YORKTOWN CSD', 'GraduationPercentageRate': '97.0', 'DropoutPercentageRate': '1.0', 'FundingPerStudent': '24424.21', 'TotalEnrolled': '3401.0', 'MalesEnrolled': '1775.0', 'FemalesEnrolled': '1626.0', 'Percentage of Minority Students': '27.8', 'HouseholdIncome': '115881.16', 'FederallyFunded': 'No'}
<cimpl.Message object at 0x7fd2b86ba440>
#1 {'DistrictName': 'CARMEL CSD', 'GraduationPercentageRate': '92.0', 'DropoutPercentageRate': '3.0', 'FundingPerStudent': '27298.65', 'TotalEnrolled': '3979.0', 'MalesEnrolled': '2034.0', 'FemalesEnrolled': '1945.0', 'Percentage of Minority Students': '37.6', 'HouseholdIncome': '110606.5', 'FederallyFunded': 'No'}
<cimpl.Message object at 0x7fd2b8f72f40>
#2 {'DistrictName': 'CARTHAGE CSD', 'GraduationPercentageRate': '89.0', 'DropoutPercentageRate': '5.0', 'FundingPerStudent': '17149.15', 'TotalEnrolled': '3065.0', 'MalesEnrolled': '1602.0', 'FemalesEnrolled': '1463.0', 'Percentage of Minority Students': '25.9', 'HouseholdIncome': '53290.64', 'FederallyFunded': 'Yes'}
<cimpl.Message object at 0x7fd2b86ba4c0>
#3 {'DistrictName': 'CASSADAGA VALLEY CSD', 'GraduationPercentageRate': '88.0', 'DropoutPercentageRate': '7.0', 'FundingPerStudent': '20862.54', 'TotalEnrolled': '799.0', 'MalesEnrolled': '422.0', 'FemalesEnrolled': '377.0', 'Percentage of Minority Students': '6.0', 'HouseholdIncome': '55387.63', 'FederallyFunded': None}
<cimpl.Message object at 0x7fd2b8f72f40>
#4 {'DistrictName': 'CATO-MERIDIAN CSD', 'GraduationPercentageRate': '89.0', 'DropoutPercentageRate': '5.0', 'FundingPerStudent': '18088.09', 'TotalEnrolled': '860.0', 'MalesEnrolled': '449.0', 'FemalesEnrolled': '411.0', 'Percentage of Minority Students': '8.1', 'HouseholdIncome': '64765.75', 'FederallyFunded': None}
<cimpl.Message object at 0x7fd2b86ba440>
#5 {'DistrictName': 'CATSKILL CSD', 'GraduationPercentageRate': '91.0', 'DropoutPercentageRate': '4.0', 'FundingPerStudent': '24088.35', 'TotalEnrolled': '1350.0', 'MalesEnrolled': '668.0', 'FemalesEnrolled': '682.0', 'Percentage of Minority Students': '32.9', 'HouseholdIncome': '58658.83', 'FederallyFunded': None}
<cimpl.Message object at 0x7fd2b8f72f40>
#6 {'DistrictName': 'CATTARAUGUS-LITTLE VALLEY CSD', 'GraduationPercentageRate': '93.0', 'DropoutPercentageRate': '4.0', 'FundingPerStudent': '19362.54', 'TotalEnrolled': '858.0', 'MalesEnrolled': '431.0', 'FemalesEnrolled': '427.0', 'Percentage of Minority Students': '6.9', 'HouseholdIncome': '52411.0', 'FederallyFunded': None}
<cimpl.Message object at 0x7fd2b86ba440>
#7 {'DistrictName': 'CAZENOVIA CSD', 'GraduationPercentageRate': '94.0', 'DropoutPercentageRate': '2.0', 'FundingPerStudent': '17708.25', 'TotalEnrolled': '1360.0', 'MalesEnrolled': '727.0', 'FemalesEnrolled': '633.0', 'Percentage of Minority Students': '6.0', 'HouseholdIncome': '69543.88', 'FederallyFunded': None}
<cimpl.Message object at 0x7fd2b8f72f40>
#8 {'DistrictName': 'CENTER MORICHES UFSD', 'GraduationPercentageRate': '93.0', 'DropoutPercentageRate': '1.0', 'FundingPerStudent': '24593.36', 'TotalEnrolled': '1513.0', 'MalesEnrolled': '788.0', 'FemalesEnrolled': '725.0', 'Percentage of Minority Students': '26.8', 'HouseholdIncome': '99102.0', 'FederallyFunded': None}
<cimpl.Message object at 0x7fd2b86ba4c0>
#9 {'DistrictName': 'CENTRAL ISLIP UFSD', 'GraduationPercentageRate': '88.0', 'DropoutPercentageRate': '1.0', 'FundingPerStudent': '21656.92', 'TotalEnrolled': '7571.0', 'MalesEnrolled': '3929.0', 'FemalesEnrolled': '3642.0', 'Percentage of Minority Students': '96.1', 'HouseholdIncome': '101402.4', 'FederallyFunded': None}
<cimpl.Message object at 0x7fd2b8f72f40>
#10 {'DistrictName': 'CENTRAL SQUARE CSD', 'GraduationPercentageRate': '87.0', 'DropoutPercentageRate': '8.0', 'FundingPerStudent': '17310.54', 'TotalEnrolled': '3580.0', 'MalesEnrolled': '1827.0', 'FemalesEnrolled': '1753.0', 'Percentage of Minority Students': '6.1', 'HouseholdIncome': '67685.73', 'FederallyFunded': None}
<cimpl.Mes

In [0]:
print(kafkaList)


[{'DistrictName': 'YORKTOWN CSD', 'GraduationPercentageRate': '97.0', 'DropoutPercentageRate': '1.0', 'FundingPerStudent': '24424.21', 'TotalEnrolled': '3401.0', 'MalesEnrolled': '1775.0', 'FemalesEnrolled': '1626.0', 'Percentage of Minority Students': '27.8', 'HouseholdIncome': '115881.16', 'FederallyFunded': 'No'}, {'DistrictName': 'CARMEL CSD', 'GraduationPercentageRate': '92.0', 'DropoutPercentageRate': '3.0', 'FundingPerStudent': '27298.65', 'TotalEnrolled': '3979.0', 'MalesEnrolled': '2034.0', 'FemalesEnrolled': '1945.0', 'Percentage of Minority Students': '37.6', 'HouseholdIncome': '110606.5', 'FederallyFunded': 'No'}, {'DistrictName': 'CARTHAGE CSD', 'GraduationPercentageRate': '89.0', 'DropoutPercentageRate': '5.0', 'FundingPerStudent': '17149.15', 'TotalEnrolled': '3065.0', 'MalesEnrolled': '1602.0', 'FemalesEnrolled': '1463.0', 'Percentage of Minority Students': '25.9', 'HouseholdIncome': '53290.64', 'FederallyFunded': 'Yes'}, {'DistrictName': 'CASSADAGA VALLEY CSD', 'GraduationPercentageRate': '88.0', 'DropoutPercentageRate': '7.0', 'FundingPerStudent': '20862.54', 'TotalEnrolled': '799.0', 'MalesEnrolled': '422.0', 'FemalesEnrolled': '377.0', 'Percentage of Minority Students': '6.0', 'HouseholdIncome': '55387.63', 'FederallyFunded': None}, {'DistrictName': 'CATO-MERIDIAN CSD', 'GraduationPercentageRate': '89.0', 'DropoutPercentageRate': '5.0', 'FundingPerStudent': '18088.09', 'TotalEnrolled': '860.0', 'MalesEnrolled': '449.0', 'FemalesEnrolled': '411.0', 'Percentage of Minority Students': '8.1', 'HouseholdIncome': '64765.75', 'FederallyFunded': None}, {'DistrictName': 'CATSKILL CSD', 'GraduationPercentageRate': '91.0', 'DropoutPercentageRate': '4.0', 'FundingPerStudent': '24088.35', 'TotalEnrolled': '1350.0', 'MalesEnrolled': '668.0', 'FemalesEnrolled': '682.0', 'Percentage of Minority Students': '32.9', 'HouseholdIncome': '58658.83', 'FederallyFunded': None}, {'DistrictName': 'CATTARAUGUS-LITTLE VALLEY CSD', 'GraduationPercentageRate': '93.0', 'DropoutPercentageRate': '4.0', 'FundingPerStudent': '19362.54', 'TotalEnrolled': '858.0', 'MalesEnrolled': '431.0', 'FemalesEnrolled': '427.0', 'Percentage of Minority Students': '6.9', 'HouseholdIncome': '52411.0', 'FederallyFunded': None}, {'DistrictName': 'CAZENOVIA CSD', 'GraduationPercentageRate': '94.0', 'DropoutPercentageRate': '2.0', 'FundingPerStudent': '17708.25', 'TotalEnrolled': '1360.0', 'MalesEnrolled': '727.0', 'FemalesEnrolled': '633.0', 'Percentage of Minority Students': '6.0', 'HouseholdIncome': '69543.88', 'FederallyFunded': None}, {'DistrictName': 'CENTER MORICHES UFSD', 'GraduationPercentageRate': '93.0', 'DropoutPercentageRate': '1.0', 'FundingPerStudent': '24593.36', 'TotalEnrolled': '1513.0', 'MalesEnrolled': '788.0', 'FemalesEnrolled': '725.0', 'Percentage of Minority Students': '26.8', 'HouseholdIncome': '99102.0', 'FederallyFunded': None}, {'DistrictName': 'CENTRAL ISLIP UFSD', 'GraduationPercentageRate': '88.0', 'DropoutPercentageRate': '1.0', 'FundingPerStudent': '21656.92', 'TotalEnrolled': '7571.0', 'MalesEnrolled': '3929.0', 'FemalesEnrolled': '3642.0', 'Percentage of Minority Students': '96.1', 'HouseholdIncome': '101402.4', 'FederallyFunded': None}, {'DistrictName': 'CENTRAL SQUARE CSD', 'GraduationPercentageRate': '87.0', 'DropoutPercentageRate': '8.0', 'FundingPerStudent': '17310.54', 'TotalEnrolled': '3580.0', 'MalesEnrolled': '1827.0', 'FemalesEnrolled': '1753.0', 'Percentage of Minority Students': '6.1', 'HouseholdIncome': '67685.73', 'FederallyFunded': None}, {'DistrictName': 'CENTRAL VALLEY CSD AT ILION-MOHAWK', 'GraduationPercentageRate': '87.0', 'DropoutPercentageRate': '5.0', 'FundingPerStudent': '15522.11', 'TotalEnrolled': '2082.0', 'MalesEnrolled': '1025.0', 'FemalesEnrolled': '1057.0', 'Percentage of Minority Students': '9.4', 'HouseholdIncome': None, 'FederallyFunded': None}, {'DistrictName': 'CHAPPAQUA CSD', 'GraduationPercentageRate': '99.0', 'DropoutPercentageRate': '0.0', 'FundingPerStudent': '29069.23', 'TotalEnrolled':

In [0]:
import pandas as pd

In [0]:
df = pd.DataFrame(kafkaList)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 10 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 DistrictName 714 non-null object
 1 GraduationPercentageRate 714 non-null object
 2 DropoutPercentageRate 714 non-null object
 3 FundingPerStudent 714 non-null object
 4 TotalEnrolled 714 non-null object
 5 MalesEnrolled 714 non-null object
 6 FemalesEnrolled 714 non-null object
 7 Percentage of Minority Students 714 non-null object
 8 HouseholdIncome 677 non-null object
 9 FederallyFunded 232 non-null object
dtypes: object(10)
memory usage: 55.9+ KB

In [0]:
finaldf = spark.createDataFrame(df)
finaldf.dtypes

Out[7]: [('DistrictName', 'string'),
 ('GraduationPercentageRate', 'string'),
 ('DropoutPercentageRate', 'string'),
 ('FundingPerStudent', 'string'),
 ('TotalEnrolled', 'string'),
 ('MalesEnrolled', 'string'),
 ('FemalesEnrolled', 'string'),
 ('Percentage of Minority Students', 'string'),
 ('HouseholdIncome', 'string'),
 ('FederallyFunded', 'string')]

In [0]:
from pyspark.sql.types import DecimalType, StringType, FloatType

output_df = finaldf \
  .withColumn("DistrictName", finaldf["DistrictName"].cast(StringType())) \
  .withColumn("GraduationPercentageRate", finaldf["GraduationPercentageRate"].cast(FloatType())) \
  .withColumn("DropoutPercentageRate", finaldf["DropoutPercentageRate"].cast(FloatType())) \
  .withColumn("FundingPerStudent", finaldf["FundingPerStudent"].cast(FloatType())) \
  .withColumn("TotalEnrolled", finaldf["TotalEnrolled"].cast(FloatType())) \
  .withColumn("MalesEnrolled", finaldf["MalesEnrolled"].cast(FloatType())) \
  .withColumn("FemalesEnrolled", finaldf["FemalesEnrolled"].cast(FloatType())) \
  .withColumn("Percentage of Minority Students", finaldf["Percentage of Minority Students"].cast(FloatType())) \
  .withColumn("HouseholdIncome", finaldf["HouseholdIncome"].cast(FloatType())) \
  .withColumn("FederallyFunded", finaldf["FederallyFunded"].cast(StringType())) 

In [0]:
output_df = output_df.withColumnRenamed("Percentage of Minority Students", "PercentageofMinorityStudents")
output_df.dtypes

Out[9]: [('DistrictName', 'string'),
 ('GraduationPercentageRate', 'float'),
 ('DropoutPercentageRate', 'float'),
 ('FundingPerStudent', 'float'),
 ('TotalEnrolled', 'float'),
 ('MalesEnrolled', 'float'),
 ('FemalesEnrolled', 'float'),
 ('PercentageofMinorityStudents', 'float'),
 ('HouseholdIncome', 'float'),
 ('FederallyFunded', 'string')]

In [0]:
database = "Capstone-Group3"
table = "dbo.nystatedistricts"
user = "stanleyperez"
password  = "Ezra!1223"
server = "gen10-data-fundamentals-22-05-sql-server.database.windows.net"

output_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("overwrite") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()